In [ ]:
import base64
import cv2
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt
import numpy as np
from keras.models import load_model

In [ ]:
b64 = "..."

In [ ]:
def img_preprocess(img):
    img = img[60:135,:,:]
    img = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
    img = cv2.GaussianBlur(img,  (3, 3), 0)
    img = cv2.resize(img, (200, 66))
    #img = img/255
    return img

In [ ]:
image = Image.open(BytesIO(base64.b64decode(b64)))
image = img_preprocess(np.asarray(image))

In [ ]:
print(image.shape)
plt.imshow(image)

In [ ]:
cv2.imwrite("img/demo.png", image)

In [ ]:
model = load_model('model.h5')

In [ ]:
# convert tensor
img_ = np.array([image])
print(img_.shape)

In [ ]:
float(model.predict(img_))